#### **Sequence-to-Sequence Model**



We explore RNN-based sequence-to-sequence (seq2seq) models to perform machine translation (MT). We will use a Vietnamese-English dataset from IWSLT'15. The task is to translate a Vietnamese sentence into English.


In [0]:
import os
import numpy as np

import matplotlib.pyplot as plt

import torch
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import util
from util import MTDataset


device = "cuda" if torch.cuda.is_available() else "cpu"
assert device == "cuda"   # use gpu!

#### **Data Preprocessing**

1. We download the dataset and place it under the "data" directory

In [0]:
util.get_dataset("en_vi_iwslt_15")

2. We do some simple data preprocessing and show some data statistics.

In [0]:
src_vocab_set = util.read_vocab_file(os.path.join("../data", "vocab.vi"))
trg_vocab_set = util.read_vocab_file(os.path.join("../data", "vocab.en"))

train_src_sentences_list = util.read_sentence_file(os.path.join("../data", "train.vi"))
train_trg_sentences_list = util.read_sentence_file(os.path.join("../data", "train.en"))
assert len(train_src_sentences_list) == len(train_trg_sentences_list)

test_src_sentences_list = util.read_sentence_file(os.path.join("../data", "tst2013.vi"))
test_trg_sentences_list = util.read_sentence_file(os.path.join("../data", "tst2013.en"))
assert len(test_src_sentences_list) == len(test_trg_sentences_list)


train_src_sentences_list, train_trg_sentences_list = util.filter_data(train_src_sentences_list, train_trg_sentences_list)
test_src_sentences_list, test_trg_sentences_list = util.filter_data(test_src_sentences_list, test_trg_sentences_list)

# We take 10% of training data as validation set.
num_val = int(len(train_src_sentences_list) * 0.1)
val_src_sentences_list = train_src_sentences_list[:num_val]
val_trg_sentences_list = train_trg_sentences_list[:num_val]
train_src_sentences_list = train_src_sentences_list[num_val:]
train_trg_sentences_list = train_trg_sentences_list[num_val:]

# Show some data stats
print("Number of training (src, trg) sentence pairs: %d", len(train_src_sentences_list))
print("Number of validation (src, trg) sentence pairs: %d", len(val_src_sentences_list))
print("Number of testing (src, trg) sentence pairs: %d", len(test_src_sentences_list))

src_vocab_set = ['<pad>'] + src_vocab_set
trg_vocab_set = ['<pad>'] + trg_vocab_set
print("Size of en vocab set (including '<pad>', '<unk>', '<s>', '</s>'): %d", len(src_vocab_set))
print("Size of vi vocab set (including '<pad>', '<unk>', '<s>', '</s>'): %d", len(trg_vocab_set))

length = [len(sent) for sent in train_src_sentences_list]
print('Training sentence avg. length: %d ', np.mean(length))
print('Training sentence length at 95-percentile: %d', np.percentile(length, 95))
print("\n")

print('Training sentence length distribution (x-axis is length range and y-axis is count)\n')
plt.hist(length, bins=5)
plt.show()

print('Example Vietnamese input: ' + str(train_src_sentences_list[0]))
print('Its target English output: ' + str(train_trg_sentences_list[0]))

#### **Encoder**

RNN seq2seq models consists of an encoder RNN and a decoder RNN.
In a vanilla seq2seq model where there is no attention mechanism between encoder and decoder, the encoder aims to compress the information contained in the entire input sequence into a single vector and pass it to decoder.

We start with implementing the encoder, which is just a simple RNN. Here we use the [GRU](https://pytorch.org/docs/stable/nn.html#gru) architecture for the RNN.

In [0]:
class Encoder(nn.Module):

  def __init__(self, input_size, hidden_size, dropout = 0.00):
    """
    Inputs: 
      - `input_size`: an int representing the RNN input size.
      - `hidden_size`: an int representing the RNN hidden size.
      - `dropout`: a float representing the dropout rate during training. Note
          that for 1-layer RNN this has no effect since dropout only applies to
          outputs of intermediate layers.
    """
    
    super().__init__()
    self.rnn = nn.GRU(input_size, hidden_size, num_layers = 1, batch_first = True, dropout = dropout, bidirectional = False)

  def forward(self, inputs, lengths):
    """
    Inputs:
      - `inputs`: a 3d-tensor of shape (batch_size, max_seq_length, embed_size)
          representing a batch of padded embedded word vectors of source
          sentences.
      - `lengths`: a 1d-tensor of shape (batch_size,) representing the sequence
          lengths of `inputs`.

    Returns:
      - `outputs`: a 3d-tensor of shape
        (batch_size, max_seq_length, hidden_size).
      - `finals`: a 3d-tensor of shape (num_layers, batch_size, hidden_size).
      Hint: `outputs` and `finals` are both standard GRU outputs. Check:
      https://pytorch.org/docs/stable/nn.html#gru
    """
    # print("inputs.shape:", inputs.shape)
    # print("lengths.shape:", lengths.shape)
    outputs = None
    finals = None
    ### Your code here!
    packed_input = pack_padded_sequence(inputs, lengths, batch_first = True, enforce_sorted = False)
    outputs, finals = self.rnn(packed_input)
    outputs, outputs_len = pad_packed_sequence(outputs, batch_first = True)

    return outputs, finals

#### **Decoder**

A RNN decoder that uses the encoder's last hidden state to initialize its initial hidden state.

In [0]:
class Decoder(nn.Module):
  """An RNN decoder without attention."""

  def __init__(self, input_size, hidden_size, dropout = 0.00):
    """
      Inputs:
        - `input_size`, `hidden_size`, and `dropout` the same as in Encoder.
    """
    super().__init__()
    self.rnn = nn.GRU(input_size, hidden_size, num_layers = 1, batch_first = True, dropout = dropout, bidirectional = False)

    
  def forward(self, inputs, encoder_finals, hidden=None, max_len=None):
    """Unroll the decoder one step at a time.

    Inputs:
      - `inputs`: a 3d-tensor of shape (batch_size, max_seq_length, embed_size)
          representing a batch of padded embedded word vectors of target
          sentences (for teacher-forcing during training).
      - `encoder_finals`: a 3d-tensor of shape
          (num_enc_layers, batch_size, hidden_size) representing the final
          encoder hidden states used to initialize the initial decoder hidden
          states.
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size) representing
          the value to be used to initialize the initial decoder hidden states.
          If None, then use `encoder_finals`.
      - `max_len`: an int representing the maximum decoding length.

    Returns:
      - `outputs`: a 3d-tensor of shape
          (batch_size, max_seq_length, hidden_size) representing the raw
          decoder outputs (before converting to a `trg_vocab_size`-dim vector).
          We will convert it later in a `Generator` below.
      - `hidden`: a 3d-tensor of shape (1, batch_size, hidden_size)
          representing the last decoder hidden state.
    """

    # The maximum number of steps to unroll the RNN.
    if max_len is None:
      max_len = inputs.size(1)

    # Initialize decoder hidden state.
    if hidden is None:
      hidden = self.init_hidden(encoder_finals)

    outputs = None
    ### Your code here!
    # print("here")
    outputs, hidden = self.rnn(inputs, hidden)

    return hidden, outputs

  def init_hidden(self, encoder_finals):
    """ Use encoder final hidden state to initialize decoder's first hidden state."""
    decoder_init_hiddens = encoder_finals

    return decoder_init_hiddens

#### **Encoder-Decoder**

We define the high level encoder-decoder class to wrap up sub-models, including encoder, decoder, generator, and src/trg embeddings.

In [0]:
class EncoderDecoder(nn.Module):
  """A standard Encoder-Decoder architecture without attention.
  """
  def __init__(self, encoder, decoder, src_embed, trg_embed, generator):
    """
    Inputs:
      - `encoder`: an `Encoder` object.
      - `decoder`: an `Decoder` object.
      - `src_embed`: an nn.Embedding object representing the lookup table for
          input (source) sentences.
      - `trg_embed`: an nn.Embedding object representing the lookup table for
          output (target) sentences.
      - `generator`: a `Generator` object. Essentially a linear mapping. See
          the next code cell.
    """
    super(EncoderDecoder, self).__init__()

    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.trg_embed = trg_embed
    self.generator = generator

  def forward(self, src_ids, trg_ids, src_lengths):
    """Take in and process masked source and target sequences.

    Inputs:
      `src_ids`: a 2d-tensor of shape (batch_size, max_seq_length) representing
        a batch of source sentences of word ids.
      `trg_ids`: a 2d-tensor of shape (batch_size, max_seq_length) representing
        a batch of target sentences of word ids.
      `src_lengths`: a 1d-tensor of shape (batch_size,) representing the
        sequence length of `src_ids`.

    Returns the decoder outputs, see the above cell.
    """
    encoder_hiddens, encoder_finals = self.encode(src_ids, src_lengths)
    del encoder_hiddens   # unused
    return self.decode(encoder_finals, trg_ids[:, :-1])

  def encode(self, src_ids, src_lengths):
    # print("src_ids.shape:", src_ids.shape)
    # print("src_lengths.shape:", src_lengths.shape)
    # print("src_ids.shape after embedding:", self.src_embed(src_ids).shape)

    return self.encoder(self.src_embed(src_ids), src_lengths)
    
  def decode(self, encoder_finals, trg_ids, decoder_hidden = None):
    return self.decoder(self.trg_embed(trg_ids), encoder_finals, decoder_hidden)

It simply projects the pre-output layer (x in the forward function below) to obtain the output layer, so that the final dimension is the target vocabulary size.

In [0]:
class Generator(nn.Module):
  """Define standard linear + softmax generation step."""
  def __init__(self, hidden_size, vocab_size):
    super().__init__()
    self.proj = nn.Linear(hidden_size, vocab_size, bias = False)

  def forward(self, x):
    return F.log_softmax(self.proj(x), dim=-1)

#### **Training and Testing**

1. Apply the dataloader to the mt dataset. The dataloader provides a convenient way to iterate through the whole dataset.

In [0]:
batch_size = 128

# note - the size of the training set may be reduced by setting a smaller value for "sampling"
train_set = util.MTDataset(train_src_sentences_list, src_vocab_set, train_trg_sentences_list, trg_vocab_set, sampling=1.)
train_data_loader = data.DataLoader(train_set, batch_size=batch_size, num_workers=8, shuffle=True)

val_set = util.MTDataset(val_src_sentences_list, src_vocab_set, val_trg_sentences_list, trg_vocab_set, sampling=1.)
val_data_loader = data.DataLoader(val_set, batch_size=batch_size, num_workers=8, shuffle=False)

2. The main functions for training, here we use perplexity to evaluate the performance of the model. Although we provide the training scripts here, we strongly encoureage you to go through and understand the procedure.

In [0]:
import math


# class SimpleLossCompute:
#   """A simple loss compute and train function."""

#   def __init__(self, generator, criterion, opt = None):
#     self.generator = generator
#     self.criterion = criterion
#     self.opt = opt

#   def __call__(self, x, y, norm):
#     x = self.generator(x)
#     loss = self.criterion(x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1))
#     loss = loss / norm

#     if self.opt is not None:  # training mode
#       loss.backward()          
#       self.opt.step()
#       self.opt.zero_grad()

#     return loss.data.item() * norm


def run_epoch(data_loader, model, loss_compute, print_every):
  """Standard Training and Logging Function"""

  total_tokens = 0
  total_loss = 0

  for i, (src_ids_BxT, src_lengths_B, trg_ids_BxL, trg_lengths_B) in enumerate(data_loader):
    # We define some notations here to help you understand the loaded tensor
    # shapes:
    #   `B`: batch size
    #   `T`: max sequence length of source sentences
    #   `L`: max sequence length of target sentences; due to our preprocessing
    #        in the beginning, `L` == `T` == 50
    # An example of `src_ids_BxT` (when B = 2):
    #   [[2, 4, 6, 7, ..., 4, 3, 0, 0, 0],
    #    [2, 8, 6, 5, ..., 9, 5, 4, 3, 0]]
    # The corresponding `src_lengths_B` would be [47, 49].
    # Note that SOS_INDEX == 2, EOS_INDEX == 3, and PAD_INDEX = 0.

    src_ids_BxT = src_ids_BxT.to(device)
    src_lengths_B = src_lengths_B.to(device)
    trg_ids_BxL = trg_ids_BxL.to(device)
    del trg_lengths_B   # unused

    # print("src_ids_BxT:", src_ids_BxT.shape)
    # print("trg_ids_BxL:", trg_ids_BxL.shape)
    # print("src_lengths_B:", src_lengths_B.shape)

    _, output = model(src_ids_BxT, trg_ids_BxL, src_lengths_B)

    loss = loss_compute(x=output, y=trg_ids_BxL[:, 1:], norm=src_ids_BxT.size(0))
    total_loss += loss
    total_tokens += (trg_ids_BxL[:, 1:] != PAD_INDEX).data.sum().item()

    if model.training and i % print_every == 0:
      print("Epoch Step: %d Loss: %f" % (i, loss / src_ids_BxT.size(0)))

    # if i == 2:
    #   print("here")
    #   break

  return math.exp(total_loss / float(total_tokens))


PAD_INDEX = 0

def train(model, num_epochs, learning_rate, print_every):
  # Set `ignore_index` as PAD_INDEX so that pad tokens won't be included when computing the loss.
  criterion = nn.NLLLoss(reduction="sum", ignore_index=PAD_INDEX)
  optim = torch.optim.Adam(model.parameters(), lr = learning_rate)

  # Keep track of dev ppl for each epoch.
  dev_ppls = []

  for epoch in range(num_epochs):
    print("epoch", epoch)

    model.train()
    train_ppl = run_epoch(data_loader = train_data_loader, model = model,
                          loss_compute = util.SimpleLossCompute(model.generator, criterion, optim),
                          print_every = print_every)
         
    model.eval()
    with torch.no_grad():      
      dev_ppl = run_epoch(data_loader = val_data_loader, model = model,
                          loss_compute = util.SimpleLossCompute(model.generator, criterion, None),
                          print_every = print_every)
      
      print("Validation perplexity: %f" % dev_ppl)
      dev_ppls.append(dev_ppl)
        
  return dev_ppls

3. The main function to perform training. First let's train the vanilla seq2seq model.

In [0]:
# Hyperparameters for contructing the encoder-decoder model.

embed_size = 256   # Each word will be represented as a `embed_size`-dim vector.
hidden_size = 256  # RNN hidden size.
dropout = 0.00

pure_seq2seq = EncoderDecoder(
  encoder = Encoder(embed_size, hidden_size, dropout = dropout),
  decoder = Decoder(embed_size, hidden_size, dropout = dropout),
  src_embed = nn.Embedding(len(src_vocab_set), embed_size),
  trg_embed = nn.Embedding(len(trg_vocab_set), embed_size),
  generator = Generator(hidden_size, len(trg_vocab_set))).to(device)

# Start training. The returned `dev_ppls` is a list of dev perplexity for each epoch.
# pure_dev_ppls = train(pure_seq2seq, num_epochs = 10, learning_rate = 1e-3, print_every = 100)
pure_dev_ppls = train(pure_seq2seq, num_epochs = 10, learning_rate = 1e-3, print_every = 800) 

4. Plot the perplexity graph.

In [0]:
util.plot_perplexity(pure_dev_ppls)

4. This is the function used to decode the model output. For simplicity, we use greedy search here.

In [0]:
def greedy_decode(model, src_ids, src_lengths, max_len):
  """Greedily decode a sentence for EncoderDecoder."""

  with torch.no_grad():
    _, encoder_finals = model.encode(src_ids, src_lengths)
    prev_y = torch.ones(1, 1).fill_(SOS_INDEX).type_as(src_ids)

  output = []
  hidden = None

  for i in range(max_len):
    with torch.no_grad():
      hidden, outputs = model.decode(encoder_finals, prev_y, hidden)
      prob = model.generator(outputs[:, -1])

    _, next_word = torch.max(prob, dim=1)
    next_word = next_word.data.item()
    output.append(next_word)
    prev_y = torch.ones(1, 1).type_as(src_ids).fill_(next_word)

  output = np.array(output)

  # Cut off everything starting from </s>.
  first_eos = np.where(output == EOS_INDEX)[0]
  if len(first_eos) > 0:
    output = output[:first_eos[0]]
  return output
  

def lookup_words(x, vocab):
  return [vocab[i] for i in x]

5. Print the top 3 examples from the data loader by applying the greedy decoder.

In [0]:
MAX_SENT_LENGTH_PLUS_SOS_EOS = 50
SOS_INDEX = 2
EOS_INDEX = 3

def print_examples(model, data_loader, n=3,
                   max_len=MAX_SENT_LENGTH_PLUS_SOS_EOS, 
                   src_vocab_set=src_vocab_set, trg_vocab_set=trg_vocab_set):
  """Prints `n` examples. Assumes batch size of 1."""

  model.eval()

  for i, (src_ids, src_lengths, trg_ids, _) in enumerate(data_loader):
    result = greedy_decode(model, src_ids.to(device), src_lengths.to(device), max_len=max_len)

    # remove <s>
    src_ids = src_ids[0, 1:]
    trg_ids = trg_ids[0, 1:]
    # remove </s> and <pad>
    src_ids = src_ids[:np.where(src_ids == EOS_INDEX)[0][0]]
    trg_ids = trg_ids[:np.where(trg_ids == EOS_INDEX)[0][0]]

    print("Example #%d" % (i + 1))
    print("Src : ", " ".join(lookup_words(src_ids, vocab=src_vocab_set)))
    print("Trg : ", " ".join(lookup_words(trg_ids, vocab=trg_vocab_set)))
    print("Pred: ", " ".join(lookup_words(result, vocab=trg_vocab_set)))
    print()

    if i == n - 1:
      break

6. Here we use the validation dataset to print examples.

In [0]:
example_set = util.MTDataset(val_src_sentences_list, src_vocab_set, val_trg_sentences_list, trg_vocab_set)
example_data_loader = data.DataLoader(val_set, batch_size=1, num_workers=1, shuffle=False)

print_examples(pure_seq2seq, example_data_loader)

7. Compute the [BLEU](https://en.wikipedia.org/wiki/BLEU) score, a standard measure to evaluate the translation results.

In [0]:
import sacrebleu
from tqdm import tqdm


def compute_BLEU(model, data_loader):
  bleu_score = []

  model.eval()
  for src_ids, src_lengths, trg_ids, _ in tqdm(data_loader):
    result = greedy_decode(model, src_ids.to(device), src_lengths.to(device),
                           max_len=MAX_SENT_LENGTH_PLUS_SOS_EOS)
    # remove <s>
    src_ids = src_ids[0, 1:]
    trg_ids = trg_ids[0, 1:]
    # remove </s> and <pad>
    src_ids = src_ids[:np.where(src_ids == EOS_INDEX)[0][0]]
    trg_ids = trg_ids[:np.where(trg_ids == EOS_INDEX)[0][0]]

    pred = " ".join(lookup_words(result, vocab=trg_vocab_set))
    targ = " ".join(lookup_words(trg_ids, vocab=trg_vocab_set))

    bleu_score.append(sacrebleu.raw_corpus_bleu([pred], [[targ]], .01).score)

  return bleu_score


test_set = MTDataset(test_src_sentences_list, src_vocab_set,
                     test_trg_sentences_list, trg_vocab_set, sampling=1.)
test_data_loader = data.DataLoader(test_set, batch_size=1, num_workers=8,
                                   shuffle=False)

print('BLEU score: %f' % (np.mean(compute_BLEU(pure_seq2seq,
                                               test_data_loader))))
# print('BLEU score: %f' % (np.mean(compute_BLEU(attn_seq2seq,
#                                                test_data_loader))))